## loading tfrecords & Parsing 

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# Parser for TFRecord
def parser(serialized_example):
    features = {
        'weight' : tf.FixedLenFeature([1], tf.float32),
        'img' : tf.FixedLenFeature([250, 550, 3], tf.int64)
    }
    parsed_feature = tf.parse_single_example(serialized_example, features)
    weight = parsed_feature['weight']
    img = parsed_feature['img']
    return weight, img

tf.reset_default_graph()

train =  "../sample_image/preprocessed/train.tfrecord"
test = "../sample_image/preprocessed/test.tfrecord"

# Dataset
BATCH_SIZE = 24

train_dataset = tf.data.TFRecordDataset(train).map(parser)
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(777)
test_dataset = tf.data.TFRecordDataset(test).map(parser)
test_dataset = test_dataset.batch(BATCH_SIZE).shuffle(777)

itr = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
weight, img = itr.get_next()

img = tf.reshape(img, [-1, 250, 550, 3]) # 차원 주의 !
img = tf.cast(img, tf.float32)

weight = tf.reshape(weight, [-1]) # 차원 주의 ! 

train_init_op = itr.make_initializer(train_dataset)
test_init_op = itr.make_initializer(test_dataset)


## Modeling 

In [6]:
def model(x, activation, dropout_prob, reuse=False):

    conv1 = tf.layers.conv2d(img, filters=16, kernel_size=7, 
                             padding='valid', activation=activation, 
                             reuse=reuse, name='conv1')
    conv1_1 = tf.layers.conv2d(conv1, filters=16, kernel_size=7, 
                             padding='valid', activation=activation, 
                             reuse=reuse, name='conv1_1')
    
    pool1 = tf.layers.max_pooling2d(conv1_1, pool_size=2, strides=2)
    
    ##############################################################
    
    conv2 = tf.layers.conv2d(pool1, filters=16, kernel_size=7,
                            padding="valid", activation=activation,
                            reuse=reuse, name='conv2')
    conv2_2 = tf.layers.conv2d(conv2, filters=16, kernel_size=7,
                            padding="valid", activation=activation,
                            reuse=reuse, name='conv2_2')
    pool2 = tf.layers.max_pooling2d(conv2_2, pool_size=2, strides=2)
    
    ##############################################################
    
    conv3 = tf.layers.conv2d(pool2, filters=32, kernel_size=3, 
                            padding="SAME", activation=activation,
                            reuse=reuse, name='conv3')
    conv3_3 = tf.layers.conv2d(conv3, filters=32, kernel_size=3, 
                            padding="SAME", activation=activation,
                            reuse=reuse, name='conv3_3')
    pool3 = tf.layers.max_pooling2d(conv3_3, pool_size=3, strides=2)

    ##############################################################
   
    flat = tf.layers.flatten(pool3)
    
    dropout1 = tf.layers.dropout(flat, rate=dropout_prob)
    fc1 = tf.layers.dense(dropout1, units=100, reuse=reuse, 
                          name='fc1')

    dropout2 = tf.layers.dropout(fc1, rate=dropout_prob)
    out = tf.layers.dense(dropout2, 1, reuse=reuse, name='out')
    
    return out 

train_out = model(img, tf.nn.relu, 0.7)
train_out = tf.reshape(train_out, [-1])
test_out = model(img, tf.nn.relu, 1, True)
test_out = tf.reshape(test_out, [-1])

loss = tf.losses.mean_squared_error(weight, train_out)
train_op = tf.train.AdagradOptimizer(1e-4).minimize(loss)

summary = tf.summary.scalar('train_loss', loss)

# test(validation) score
pred = test_out
mse = tf.metrics.mean_squared_error(weight, pred)
    
merged = tf.summary.merge_all()

saver = tf.train.Saver()

In [7]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(train_init_op)
    sess.run(loss)
    sess.run(train_op)
    

OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,1], [?,250,550,3]], output_types=[DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]

Caused by op 'IteratorGetNext', defined at:
  File "/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/ipyke

In [9]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
#     writer = tf.summary.FileWriter('../logs/', sess.graph)
    loss_graph = []
    mse_graph = []
    
    for epoch in range(100):
        sess.run(train_init_op)
        for train_step in range(9999999):
            try :
                print("start")
                _ = sess.run(train_op)
#                 _, _loss, _summ = sess.run([train_op, loss, summary])
                print(_loss)
                loss_graph.append(_loss)
                writer.add_summary(_summ, train_step)
                    
            except tf.errors.OutOfRangeError :
                break
                
                
        sess.run(test_init_op)
        for test_step in range(99999999):
                try:
                    _mse = sess.run(mse)
                    mse_graph.append(_mse)
                except tf.errors.OutOfRangeError:
                    break
        print('epochs : {}, train_loss : {}, val_mse : {}'.format(epoch, _loss, _mse[0]))
        saver.save(sess, "../logs/cnn{}".format(epoch))

start


NameError: name '_loss' is not defined

In [ ]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
#     writer = tf.summary.FileWriter('../logs/', sess.graph)
    loss_graph = []
    mse_graph = []
    
    for epoch in range(100):
        sess.run(train_init_op)
        for train_step in range(9999999):
            try :
                _, _loss, _summ = sess.run([train_op, loss, summary])
                loss_graph.append(_loss)
                writer.add_summary(_summ, train_step)
                    
            except tf.errors.OutOfRangeError :
                print(error)
                break
                
                
        sess.run(test_init_op)
        for test_step in range(99999999):
                try:
                    _mse = sess.run(mse)
                    mse_graph.append(_mse)
                except tf.errors.OutOfRangeError:
                    break
        print('epochs : {}, train_loss : {}, val_mse : {}'.format(epoch, _loss, _mse[0]))
        saver.save(sess, "../logs/cnn{}".format(epoch))

### DenseNet Implementation 

> - link(https://github.com/YixuanLi/densenet-tensorflow/blob/master/cifar10-densenet.py#L25)
> - PAPER(https://arxiv.org/pdf/1608.06993.pdf)
> - 

In [ ]:
# 윤철이 형 버전 

import tensorflow as tf
import numpy as np


class DenseNet:
    def batch_norm_relu(self, inputs, is_training, reuse, name):
        bn = tf.layers.batch_normalization(inputs, 
                                           training=is_training, 
                                           reuse=reuse, 
                                           name=name)
        outputs = tf.nn.relu(bn)
        
        return outputs
    
    
    def initial_conv(self, inputs, reuse=False):
        l = tf.layers.conv2d(inputs=inputs,
                             filters=16,
                             kernel_size=3,
                             strides=2,
                             padding='SAME',
                             name='init_conv',
                             reuse=reuse)
        
        return l

    
    def composite_layer(self, inputs, keep_prob, name, is_training=True, reuse=False):
        l = inputs
        l = self.batch_norm_relu(l, is_training, reuse, name=name+'_bn1')
        l = tf.layers.conv2d(l, 4 * 12, 1, 1, 
                             padding='SAME', name=name+'_conv1', reuse=reuse)
            
        l = self.batch_norm_relu(l, is_training, reuse, name=name+'_bn2')
        
        l = tf.layers.conv2d(l, 12, 3, 1, 
                             padding='SAME', name=name+'_conv2', reuse=reuse)
        
        l = tf.layers.dropout(l, keep_prob, training=is_training)
        
        return tf.concat([inputs, l], axis=3) 


    def transition_layer(self, inputs, name, is_training=True, reuse=False):
        shape = inputs.get_shape().as_list()
        n_filters = int(shape[3] * 0.5)
        
        l = self.batch_norm_relu(inputs, is_training, reuse, name=name + '_bn')
        l = tf.layers.conv2d(l, n_filters, 1, 1, padding='SAME', name=name + '_conv', reuse=reuse)
        l = tf.layers.average_pooling2d(l, 2, 2, name='pool')

        return l
    
    def dense_net(self, inputs, keep_prob=0.2, is_training=True, reuse=False):
        l = self.initial_conv(inputs=inputs, reuse=reuse)
        
        with tf.variable_scope('block1') as scope:
            for i in range(6):
                l = self.composite_layer(l, 
                                         keep_prob, 
                                         name='dense_layer{}'.format(i), 
                                         is_training=is_training,
                                         reuse=reuse)
                
            l = self.transition_layer(l, 
                                     name='transition1',
                                     is_training=is_training,
                                     reuse=reuse)
                
        with tf.variable_scope('block2') as scope:
            for i in range(12):
                l = self.composite_layer(l, 
                                         keep_prob, 
                                         name='dense_layer{}'.format(i),
                                         is_training=is_training,
                                         reuse=reuse)

            l = self.transition_layer(l, 
                                     name='transition2',
                                     is_training=is_training,
                                     reuse=reuse)

        with tf.variable_scope('block3') as scope:
            for i in range(24):
                l = self.composite_layer(l, 
                                         keep_prob, 
                                         name='dense_layer{}'.format(i),
                                         is_training=is_training,
                                         reuse=reuse)
                
            l = self.transition_layer(l, 
                                     name='transition3',
                                     is_training=is_training,
                                     reuse=reuse)

        with tf.variable_scope('block4') as scope:
            for i in range(16):
                l = self.composite_layer(l, 
                                         keep_prob, 
                                         name='dense_layer{}'.format(i),
                                         is_training=is_training,
                                         reuse=reuse)
                
        return l
    
    
    def get_logits(self, inputs, is_training=True, reuse=False):
        l = self.dense_net(inputs, keep_prob=0.2, is_training=is_training, reuse=reuse)
        
        outputs = self.batch_norm_relu(l, is_training, reuse, name='last_bn')

        shape = outputs.get_shape().as_list()
        
        pool_size = (shape[1], shape[2])
        outputs= tf.layers.average_pooling2d(outputs, pool_size=pool_size, strides=1, padding='VALID')
        
        outputs = tf.layers.flatten(outputs)
        outputs = tf.layers.dense(outputs, 12, name='final_dense', reuse=reuse)
        
        return outputs
    
    
# class CnnLstm:
#     def __init__(self):
#         self.num_classes = 12
#         self.num_filters = [8, 16, 32, 32]
#         self.filter_sizes = [7, 3, 3, 3]
#         self.pool_sizes = [2, 2, 1, 1]
#         self.cnn_dropout_keep_prob = [0, 0.3, 0.4, 0.4]
#         self.fc_hidden_units = [1028, 512, 256]
#         self.fc_dropout_keep_prob = [0.2, 0.3, 0.35]
#         self.lstm_n_hiddens = [512]
#         self.lstm_dropout_keep_prob = [0.5]
#         self.idx_convolutional_layers = range(1, len(self.filter_sizes) + 1)
#         self.idx_fc_layers = range(1, len(self.fc_hidden_units) + 1)
#         self.idx_lstm_layers = range(1, len(self.lstm_n_hiddens) + 1)
        
        
#     def convolutional_layer(self, inputs, is_training=True, reuse=False):
#         l = inputs
        
#         for i, num_filter, filter_size, pool_size, keep_prob in zip(self.idx_convolutional_layers,
#                                                                     self.num_filters,
#                                                                     self.filter_sizes,
#                                                                     self.pool_sizes,
#                                                                     self.cnn_dropout_keep_prob):
#             l = tf.layers.conv2d(l, 
#                                  filters=num_filter, 
#                                  kernel_size=filter_size, 
#                                  strides=1, 
#                                  padding="SAME", 
#                                  name="conv"+str(i),
#                                  reuse=reuse)
            
#             l = tf.layers.batch_normalization(l, training=is_training, name="conv_bn"+str(i), reuse=reuse)
#             l = tf.nn.relu(l, name="conv_relu"+str(i))
#             l = tf.layers.dropout(l, rate=keep_prob, training=is_training, name="conv_dropout"+str(i))

#             if pool_size != 1:
#                 l = tf.layers.max_pooling2d(l, pool_size=pool_size, strides=pool_size, padding="SAME")
                
#         return l
        
    
    def fc_layer(self, inputs, is_training=True, reuse=False):
        l = inputs
        
        for i, units, keep_prob in zip(self.idx_fc_layers, self.fc_hidden_units, self.fc_dropout_keep_prob):
            l = tf.layers.dense(inputs, units=units, reuse=reuse, name="fc"+str(i))
            l = tf.layers.batch_normalization(l, training=is_training, name="fc_bn"+str(i), reuse=reuse)
            l = tf.nn.relu(l, name="fc_relu"+str(i))
            l = tf.layers.dropout(l, rate=keep_prob, training=is_training, name="fc_dropout"+str(i))
            
        return l
  

    def lstm_layer(self, inputs, is_training=True, reuse=False):
        if is_training:
            keep_probs = [0.5]
            
        else:
            keep_probs = [1]
            
        cell = tf.nn.rnn_cell.BasicLSTMCell(512, reuse=reuse)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_probs[0])
        
        outputs, states = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
        outputs = tf.transpose(outputs, [1, 0, 2])
        outputs = outputs[-1]
        
        return outputs
 

    def get_reshaped_cnn_to_rnn(self, inputs):
        shape = inputs.get_shape().as_list() 
        inputs = tf.transpose(inputs, [0, 2, 1, 3])
        reshaped_inputs = tf.reshape(inputs, [-1, shape[2], shape[1] * shape[3]])
        
        return reshaped_inputs
  

    def get_logits(self, inputs, is_training=True, reuse=False):
        with tf.variable_scope("conv_layers") as scope:
            l = inputs
            l = self.convolutional_layer(l, is_training, reuse)
            
        with tf.variable_scope("lstm_layers") as scope:
            reshaped_l = self.get_reshaped_cnn_to_rnn(l)
            
            l = self.lstm_layer(reshaped_l, is_training, reuse)
            
        with tf.variable_scope("fc_layers") as scope:
            l = tf.layers.flatten(l)
            l = self.fc_layer(l, is_training, reuse)
                
        output = tf.layers.dense(l, units=self.num_classes, reuse=reuse, name='out')
            
        return output
    

def train_parser(serialized_example):
    features = {
        "spectrum": tf.FixedLenFeature([12800], tf.float32),
        "label": tf.FixedLenFeature([12], tf.int64)
    }

    parsed_feature = tf.parse_single_example(serialized_example, features)

    spec = parsed_feature['spectrum']
    label = parsed_feature['label']

    return spec, label
        
    
def test_parser(serialized_example):
    features = {
        "spectrum": tf.FixedLenFeature([12800], tf.float32),
    }

    parsed_feature = tf.parse_single_example(serialized_example, features)

    spec = parsed_feature['spectrum']

    return spec

In [1]:
import tensorflow as tf
from tflearn.layers.conv import global_avg_pool 
from tensorflow.contrib.layers import batch_norm, flatten
from tensorflow.contrib.layers import xavier_initializer # weight 
from tensorflow.contrib.framework import arg_scope
from cifar10 import *

# Hyperparameter
growth_k = 24 # growth = 
nb_block = 2 # how many (dense block + Transition Layer) ?
init_learning_rate = 1e-4
epsilon = 1e-4 # AdamOptimizer epsilon
dropout_rate = 0.2

# Momentum Optimizer will use
nesterov_momentum = 0.9
weight_decay = 1e-4

# Label & batch_size
batch_size = 64

iteration = 782
# batch_size * iteration = data_set_number

test_iteration = 10

total_epochs = 300

def conv_layer(input, filter, kernel, stride=1, layer_name="conv"):
    with tf.name_scope(layer_name):
        network = tf.layers.conv2d(inputs=input, use_bias=False, filters=filter, kernel_size=kernel, strides=stride, padding='SAME')
        return network

def Global_Average_Pooling(x, stride=1):
    """
    width = np.shape(x)[1]
    height = np.shape(x)[2]
    pool_size = [width, height]
    return tf.layers.average_pooling2d(inputs=x, pool_size=pool_size, strides=stride) # The stride value does not matter
    It is global average pooling without tflearn
    """

    return global_avg_pool(x, name='Global_avg_pooling')
    # But maybe you need to install h5py and curses or not


def Batch_Normalization(x, training, scope):
    with arg_scope([batch_norm],
                   scope=scope,
                   updates_collections=None,
                   decay=0.9,
                   center=True,
                   scale=True,
                   zero_debias_moving_mean=True) :
        return tf.cond(training,
                       lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                       lambda : batch_norm(inputs=x, is_training=training, reuse=True))

def Drop_out(x, rate, training) :
    return tf.layers.dropout(inputs=x, rate=rate, training=training)

def Relu(x):
    return tf.nn.relu(x)

def Average_pooling(x, pool_size=[2,2], stride=2, padding='VALID'):
    return tf.layers.average_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)


def Max_Pooling(x, pool_size=[3,3], stride=2, padding='VALID'):
    return tf.layers.max_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)

def Concatenation(layers) :
    return tf.concat(layers, axis=3)

def Linear(x) :
    return tf.layers.dense(inputs=x, units=class_num, name='linear')

def Evaluate(sess):
    test_acc = 0.0
    test_loss = 0.0
    test_pre_index = 0
    add = 1000

    for it in range(test_iteration):
        test_batch_x = test_x[test_pre_index: test_pre_index + add]
        test_batch_y = test_y[test_pre_index: test_pre_index + add]
        test_pre_index = test_pre_index + add

        test_feed_dict = {
            x: test_batch_x,
            label: test_batch_y,
            learning_rate: epoch_learning_rate,
            training_flag: False
        }

        loss_, acc_ = sess.run([cost, accuracy], feed_dict=test_feed_dict)

        test_loss += loss_ / 10.0
        test_acc += acc_ / 10.0

    summary = tf.Summary(value=[tf.Summary.Value(tag='test_loss', simple_value=test_loss),
                                tf.Summary.Value(tag='test_accuracy', simple_value=test_acc)])

    return test_acc, test_loss, summary

class DenseNet():
    def __init__(self, x, nb_blocks, filters, training):
        self.nb_blocks = nb_blocks
        self.filters = filters
        self.training = training
        self.model = self.Dense_net(x)


    def bottleneck_layer(self, x, scope):
        # print(x)
        with tf.name_scope(scope):
            x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
            x = Relu(x)
            x = conv_layer(x, filter=4 * self.filters, kernel=[1,1], layer_name=scope+'_conv1')
            x = Drop_out(x, rate=dropout_rate, training=self.training)

            x = Batch_Normalization(x, training=self.training, scope=scope+'_batch2')
            x = Relu(x)
            x = conv_layer(x, filter=self.filters, kernel=[3,3], layer_name=scope+'_conv2')
            x = Drop_out(x, rate=dropout_rate, training=self.training)

            # print(x)

            return x

    def transition_layer(self, x, scope):
        with tf.name_scope(scope):
            x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
            x = Relu(x)
            x = conv_layer(x, filter=self.filters, kernel=[1,1], layer_name=scope+'_conv1')
            x = Drop_out(x, rate=dropout_rate, training=self.training)
            x = Average_pooling(x, pool_size=[2,2], stride=2)

            return x

    def dense_block(self, input_x, nb_layers, layer_name):
        with tf.name_scope(layer_name):
            layers_concat = list()
            layers_concat.append(input_x)

            x = self.bottleneck_layer(input_x, scope=layer_name + '_bottleN_' + str(0))

            layers_concat.append(x)

            for i in range(nb_layers - 1):
                x = Concatenation(layers_concat)
                x = self.bottleneck_layer(x, scope=layer_name + '_bottleN_' + str(i + 1))
                layers_concat.append(x)

            x = Concatenation(layers_concat)

            return x

    def Dense_net(self, input_x):
        x = conv_layer(input_x, filter=2 * self.filters, kernel=[7,7], stride=2, layer_name='conv0')
        # x = Max_Pooling(x, pool_size=[3,3], stride=2)


        """
        for i in range(self.nb_blocks) :
            # 6 -> 12 -> 48
            x = self.dense_block(input_x=x, nb_layers=4, layer_name='dense_'+str(i))
            x = self.transition_layer(x, scope='trans_'+str(i))
        """




        x = self.dense_block(input_x=x, nb_layers=6, layer_name='dense_1')
        x = self.transition_layer(x, scope='trans_1')

        x = self.dense_block(input_x=x, nb_layers=12, layer_name='dense_2')
        x = self.transition_layer(x, scope='trans_2')

        x = self.dense_block(input_x=x, nb_layers=48, layer_name='dense_3')
        x = self.transition_layer(x, scope='trans_3')

        x = self.dense_block(input_x=x, nb_layers=32, layer_name='dense_final')



        # 100 Layer
        x = Batch_Normalization(x, training=self.training, scope='linear_batch')
        x = Relu(x)
        x = Global_Average_Pooling(x)
        x = flatten(x)
        x = Linear(x)


        # x = tf.reshape(x, [-1, 10])
        return x



train_x, train_y, test_x, test_y = prepare_data()
train_x, test_x = color_preprocessing(train_x, test_x)

# image_size = 32, img_channels = 3, class_num = 10 in cifar10
x = tf.placeholder(tf.float32, shape=[None, image_size, image_size, img_channels])
label = tf.placeholder(tf.float32, shape=[None, class_num])

training_flag = tf.placeholder(tf.bool)


learning_rate = tf.placeholder(tf.float32, name='learning_rate')

logits = DenseNet(x=x, nb_blocks=nb_block, filters=growth_k, training=training_flag).model
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits))

"""
l2_loss = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=nesterov_momentum, use_nesterov=True)
train = optimizer.minimize(cost + l2_loss * weight_decay)
In paper, use MomentumOptimizer
init_learning_rate = 0.1
but, I'll use AdamOptimizer
"""

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=epsilon)
train = optimizer.minimize(cost)


correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver(tf.global_variables())

with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state('./model')
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())

    summary_writer = tf.summary.FileWriter('./logs', sess.graph)

    epoch_learning_rate = init_learning_rate
    for epoch in range(1, total_epochs + 1):
        if epoch == (total_epochs * 0.5) or epoch == (total_epochs * 0.75):
            epoch_learning_rate = epoch_learning_rate / 10

        pre_index = 0
        train_acc = 0.0
        train_loss = 0.0


        for step in range(1, iteration + 1):
            if pre_index+batch_size < 50000 :
                batch_x = train_x[pre_index : pre_index+batch_size]
                batch_y = train_y[pre_index : pre_index+batch_size]
            else :
                batch_x = train_x[pre_index : ]
                batch_y = train_y[pre_index : ]

            batch_x = data_augmentation(batch_x)

            train_feed_dict = {
                x: batch_x,
                label: batch_y,
                learning_rate: epoch_learning_rate,
                training_flag : True
            }

            _, batch_loss = sess.run([train, cost], feed_dict=train_feed_dict)
            batch_acc = accuracy.eval(feed_dict=train_feed_dict)

            train_loss += batch_loss
            train_acc += batch_acc
            pre_index += batch_size

            if step == iteration :
                train_loss /= iteration # average loss
                train_acc /= iteration # average accuracy

                train_summary = tf.Summary(value=[tf.Summary.Value(tag='train_loss', simple_value=train_loss),
                                                  tf.Summary.Value(tag='train_accuracy', simple_value=train_acc)])

                test_acc, test_loss, test_summary = Evaluate(sess)

                summary_writer.add_summary(summary=train_summary, global_step=epoch)
                summary_writer.add_summary(summary=test_summary, global_step=epoch)
                summary_writer.flush()

                line = "epoch: %d/%d, train_loss: %.4f, train_acc: %.4f, test_loss: %.4f, test_acc: %.4f \n" % (
                    epoch, total_epochs, train_loss, train_acc, test_loss, test_acc)
                print(line)

                with open('logs.txt', 'a') as f :
                    f.write(line)



        saver.save(sess=sess, save_path='./model/dense.ckpt')


/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


KeyboardInterrupt: 